In [1]:
from helpme import *

[x₀  x₁  x₂  ẋ₀  ẋ₁  ẋ₂]


In [2]:
# create state vector with 4 components and derivatives up to order 2
# and make available in global namespace
xx = gen_state(2,3)
xx.T

[x₁  x₂  ẋ₁  ẋ₂  ẍ₁  ẍ₂  ẍ̇₁  ẍ̇₂]

Matrix to investigate:

$\mathbf{A}(\lambda)=%\mathbf{A}_0+\mathbf{A}_1\lambda=
\begin{pmatrix}
    -\dot{x}_2\lambda & -\dot{x}_1\lambda & \lambda\\
    \dot{x}_2 & 0 & -1\\
    0 & 1 & 0
\end{pmatrix}
=
\underbrace{
\begin{pmatrix}
    0 & 0 & 0\\
    \dot{x}_2 & 0 & -1\\
    0 & 1 & 0
\end{pmatrix}}_{=\mathbf{A}_0}
+
\underbrace{
\begin{pmatrix}
    -\dot{x}_2 & -\dot{x}_1 & 0\\
    0 & 0 & 0\\
    0 & 0 & 0
\end{pmatrix}}_{=\mathbf{A}_1}\lambda
$

In [3]:
A0 = sp.Matrix([
    [0,0,0],
    [xdot2,0,-1],
    [0,1,0]])
A1 = sp.Matrix([
    [-xdot2,-xdot1,1],
    [0,0,0],
    [0,0,0]])
A0,A1

⎛⎡ 0   0  0 ⎤, ⎡-ẋ₂  -ẋ₁  1⎤⎞
⎜⎢          ⎥  ⎢             ⎥⎟
⎜⎢ẋ₂  0  -1⎥  ⎢ 0     0    0⎥⎟
⎜⎢          ⎥  ⎢             ⎥⎟
⎝⎣ 0   1  0 ⎦  ⎣ 0     0    0⎦⎠

In [4]:
# manually:
# st.rnd_number_rank(spread(A0,A1,beta=0)) \
# == st.rnd_number_rank(spread_aug(A0,A1,beta=0)) \
# and st.rnd_number_rank(spread(A0,A1,beta=0)) == spread(A0,A1,beta=0).shape[0]

In [5]:
# st.rnd_number_rank(spread(A0,A1,beta=1)) \
# == st.rnd_number_rank(spread_aug(A0,A1,beta=1)) \
# and st.rnd_number_rank(spread(A0,A1,beta=1)) == spread(A0,A1,beta=1).shape[0]

In [6]:
# automated:
# returns degree beta of inverse (-1 => not unimodular)
beta_ = is_unimodular(A0,A1)
if (beta_==-1):
    print("A is not unimodular")
else:
    print("A is unimodular")

A is unimodular


In [8]:
sA1 = Top(A0,A1,beta=beta_)
sA1

⎡   0      0    0    -1.0⋅ẋ₂  -1.0⋅ẋ₁  1.0      0         0       0 ⎤
⎢                                                                     ⎥
⎢1.0⋅ẋ₂   0   -1.0     0         0       0       0         0       0 ⎥
⎢                                                                     ⎥
⎢   0     1.0   0       0         0       0       0         0       0 ⎥
⎢                                                                     ⎥
⎢   0      0    0    -1.0⋅ẍ₂  -1.0⋅ẍ₁   0    -1.0⋅ẋ₂  -1.0⋅ẋ₁  1.0⎥
⎢                                                                     ⎥
⎢1.0⋅ẍ₂   0    0    1.0⋅ẋ₂      0      -1.0     0         0       0 ⎥
⎢                                                                     ⎥
⎣   0      0    0       0        1.0      0       0         0       0 ⎦

In [9]:
sA1_rpinv = sp.simplify(right_pseudo_inverse(sA1))
sA1_rpinv

⎡  1.0                      1.0      1.0⋅ẋ₁  ⎤
⎢  ───     0     0    0     ───      ───────  ⎥
⎢  ẍ₂                      ẍ₂        ẍ₂    ⎥
⎢                                             ⎥
⎢   0      0    1.0   0      0          0     ⎥
⎢                                             ⎥
⎢1.0⋅ẋ₂                  1.0⋅ẋ₂  1.0⋅ẋ₁⋅ẋ₂⎥
⎢───────  -1.0   0    0   ───────  ───────────⎥
⎢  ẍ₂                      ẍ₂        ẍ₂    ⎥
⎢                                             ⎥
⎢   0      0     0    0      0          0     ⎥
⎢                                             ⎥
⎢   0      0     0    0      0         1.0    ⎥
⎢                                             ⎥
⎢  1.0     0     0    0      0       1.0⋅ẋ₁  ⎥
⎢                                             ⎥
⎢   0      0     0    0      0          0     ⎥
⎢                                             ⎥
⎢   0      0     0    0      0          0     ⎥
⎢                                             ⎥
⎣   0      0     0   1.0     0       1.0

In [10]:
I, O = sp.eye(3), sp.zeros(3,6)
IO = st.col_stack(I,O)
B_ = IO*sA1_rpinv
B_

⎡  1.0                    1.0      1.0⋅ẋ₁  ⎤
⎢  ───     0     0   0    ───      ───────  ⎥
⎢  ẍ₂                    ẍ₂        ẍ₂    ⎥
⎢                                           ⎥
⎢   0      0    1.0  0     0          0     ⎥
⎢                                           ⎥
⎢1.0⋅ẋ₂                1.0⋅ẋ₂  1.0⋅ẋ₁⋅ẋ₂⎥
⎢───────  -1.0   0   0  ───────  ───────────⎥
⎣  ẍ₂                    ẍ₂        ẍ₂    ⎦

In [11]:
B0 = st.col_select(B_, 0,1,2)
B1 = st.col_select(B_, 3,4,5)
B0, B1 # B0+B1*s

⎛⎡  1.0             ⎤  ⎡     1.0      1.0⋅ẋ₁  ⎤⎞
⎜⎢  ───     0     0 ⎥, ⎢0    ───      ───────  ⎥⎟
⎜⎢  ẍ₂             ⎥  ⎢     ẍ₂        ẍ₂    ⎥⎟
⎜⎢                  ⎥  ⎢                       ⎥⎟
⎜⎢   0      0    1.0⎥  ⎢0     0          0     ⎥⎟
⎜⎢                  ⎥  ⎢                       ⎥⎟
⎜⎢1.0⋅ẋ₂           ⎥  ⎢   1.0⋅ẋ₂  1.0⋅ẋ₁⋅ẋ₂⎥⎟
⎜⎢───────  -1.0   0 ⎥  ⎢0  ───────  ───────────⎥⎟
⎝⎣  ẍ₂             ⎦  ⎣     ẍ₂        ẍ₂    ⎦⎠

Unimodular Inverse of $\mathbf{A}(\lambda):$

$\mathbf{B}(\lambda)
=\mathbf{B}_0+\mathbf{B}_1\lambda
=
\begin{pmatrix}
    \frac{1}{\ddot{x}_2} & \frac{1}{\ddot{x}_2}\lambda & \frac{\dot{x}_1}{\ddot{x}_2}\lambda\\
    0 & 0 & 1\\
    \frac{\dot{x}_2}{\ddot{x}_2} & \frac{\dot{x}_2}{\ddot{x}_2}\lambda-1 & \frac{\dot{x}_1\dot{x}_2}{\ddot{x}_2}\lambda
\end{pmatrix}
$